In [1]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.append(os.path.abspath("../"))  # Adds 'src' to the module search path
sys.path.append(os.path.abspath("../src"))  # Adds 'src' to the module search path

In [2]:
from utils_data import create_model_dataset
from read_data import apply_index_file

In [3]:
np.random.seed(42)
RAW_DATASET = "../raw_data/massive.parquet"
DATA = "../data/"
TRAIN = "../data/train_subsets/"
TEST = "../data/test/"
os.makedirs(DATA, exist_ok=True)
os.makedirs(TRAIN, exist_ok=True)
os.makedirs(TEST, exist_ok=True)

# Preprocess

In [5]:
create_model_dataset(RAW_DATASET,f"{DATA}dataset.parquet")

# Split

In [7]:
df = pd.read_parquet(f"{DATA}dataset.parquet",engine="pyarrow")

In [8]:
sequences_dataset = df.groupby('dataset')['modified_sequence'].count()

In [9]:
all_datasets = df['dataset'].unique()
test_datasets = ['MSV000080274']
val_datasets = ['MSV000079550']
train_datasets = [d for d in all_datasets if d not in test_datasets + val_datasets]

In [10]:
test_df = df[df['dataset'].isin(test_datasets)].sample(350000, random_state=42) 
val_df = df[df['dataset'].isin(val_datasets)].sample(350000, random_state=42) 

In [11]:
test_sequences_with_intersection = set(test_df['modified_sequence'])
val_sequences_with_intersection = set(val_df['modified_sequence'])
common_sequences = test_sequences_with_intersection.intersection(val_sequences_with_intersection)
common_sequences_list = list(common_sequences)

In [12]:
np.random.shuffle(common_sequences_list)
half_point = len(common_sequences_list) // 2
test_keep_sequences = set(common_sequences_list[:half_point])
val_keep_sequences = set(common_sequences_list[half_point:])

test_df = test_df[~test_df['modified_sequence'].isin(val_keep_sequences)]
val_df = val_df[~val_df['modified_sequence'].isin(test_keep_sequences)]
print(f"test dataset size: {len(test_df)}")
print(f"validation dataset size: {len(val_df)}")
test_sequences = set(test_df["modified_sequence"].unique())
val_sequences = set(val_df["modified_sequence"].unique())
test_val_sequences = test_sequences | val_sequences

test dataset size: 242478
validation dataset size: 251021


In [13]:
train_df = df[df['dataset'].isin(train_datasets)]
train_df = train_df[~train_df['modified_sequence'].isin(test_val_sequences)]
train_sequences = set(train_df["modified_sequence"].unique())

In [14]:
if not (test_sequences & val_sequences or test_sequences & train_sequences or val_sequences & train_sequences):
    print("No sequence overlap between the sets")
else:
    print("There is an overlap in sequences between the sets.")

No sequence overlap between the sets


In [15]:
train_df.index.to_series().to_csv(f"{DATA}/train.csv", index=False,header=False)
test_df.index.to_series().to_csv(f"{DATA}/test.csv", index=False,header=False)
val_df.index.to_series().to_csv(f"{DATA}/val.csv", index=False,header=False)

## MSV000080814 train indices

In [17]:
df = pd.read_parquet(f"{DATA}dataset.parquet",engine="pyarrow")
train_df = apply_index_file(df,f"{DATA}train.csv")

In [18]:
MSV80814_indices = train_df[train_df['dataset']=="MSV000080814"].index.to_series()
MSV80814_indices.to_csv(f'{TRAIN}MSV000080814.csv', index=False,header=False)

## increasing train indices

In [20]:
df = pd.read_parquet(f"{DATA}dataset.parquet",engine="pyarrow")
train_df = apply_index_file(df,f"{DATA}train.csv")

In [21]:
unique_filenames = train_df['filename'].unique()
total_files = len(unique_filenames)
step_size = int(total_files * 0.1)  # Calculate 10% of total files

# Create steps at 10% increments up to 90%
steps = []
for i in range(1, 10):  # 10%, 20%, 30%, ... 90% (stopping before 100%)
    steps.append(i * step_size)

# Add 100% as the final step
steps.append(total_files)
for x in steps:
    subset = unique_filenames[:x]
    train_df[train_df['filename'].isin(subset)].index.to_series().to_csv(
        f'{TRAIN}runs_{x}.csv', index=False, header=False
    )

## Small test sample

In [23]:
df = pd.read_parquet(f"{DATA}dataset.parquet",engine="pyarrow")
df_test = df.head(200)
df_test.loc[0, 'filename'] = 'x'
df_test.loc[1, 'filename'] = 'y'
df_test.loc[2, 'filename'] = 'z'
train = pd.Series(range(0, 100))     
val = pd.Series(range(100, 150))  
test = pd.Series(range(150, 200))   
# Save each to a CSV
train.to_csv(f"{TEST}train.csv", index=False, header=False)
val.to_csv(f"{TEST}val.csv", index=False, header=False)
test.to_csv(f"{TEST}test.csv", index=False, header=False)
df_test.to_parquet(f"{TEST}dataset.parquet")